In [2]:
import os
import random

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip



7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://storage.googleapis.com]87[Files: 0  Bytes: 0  [0 B/s] Re]87Saving '101_food_classes_10_percent.zip'
87101_food_classes_10_   0% [>                             ]  276.52K    --.-KB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87[Files: 0  Bytes: 0  [0 B/s] Re]87101_food_classes_10_   0% [>                             ]    9.93M    9.66MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87101_food_classes_10_   1% [>                             ]   21.35M   10.54MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87101_food_classes_10_   2% [>                             ]   32.54M   10.75MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87101_food_classes_10_   2% [>                             ]   43.28M   10.75MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87101_food_classes_10_   3% [>                             ]   53.90M   10.72MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87101_food_classes_10_   4% [>                             ]   63.24M   10.49MB/s87[Files: 0 

In [3]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [4]:
import zipfile

zipfile = zipfile.ZipFile("101_food_classes_10_percent.zip")
zipfile.extractall()
zipfile.close()

KeyboardInterrupt: 

**Create a model checkpoint callback**

In [ ]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint.weights.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [ ]:
data_augmentation = tf.keras.models.Sequential([
  tf.keras.layers.RandomFlip("horizontal"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomHeight(0.2),
])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

train_data_without_augmentation = ImageDataGenerator(
    rescale=1./255
)

test_data_without_augmentation = ImageDataGenerator(
    rescale=1./255
)

train_data = train_data_without_augmentation.flow_from_directory(
    train_dir,
    target_size = IMAGE_SHAPE,
    class_mode='categorical',
    batch_size=32,
)


test_data = test_data_without_augmentation.flow_from_directory(
    test_dir,
    target_size = IMAGE_SHAPE,
    class_mode='categorical',
    batch_size=32,
)

classes = train_data.class_indices
classes = {value: key for key, value in classes.items()}

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import (Input, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

data_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomZoom(0.3),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomContrast(0.2)
])

base_model = EfficientNetB3(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

inputs = Input(shape=(224, 224, 3), name="input_layer")

x = tf.keras.layers.Resizing(224, 224)(inputs)
x = data_augmentation(x)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

outputs = Dense(101, activation="softmax", name="output_layer")(x)

model = Model(inputs, outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1),
  ]

In [ ]:
history_1 = model.fit(train_data,
                        epochs=10,
                        steps_per_epoch=len(train_data),
                        validation_data=test_data,
                        validation_steps=len(test_data),
                        callbacks=callbacks
                        )

In [ ]:
results_feature_extraction_model = model.evaluate(test_data)
results_feature_extraction_model

In [ ]:
model.save("101_classes_10_percent_data_model_1.keras")

In [ ]:
y_pred = model.predict(test_data)

In [ ]:
pred_classes = y_pred.argmax(axis=1)

pred_classes[:10]